In [2]:
from bs4 import BeautifulSoup
import urllib3
import re
import time
import pandas as pd
from pandas import Series, DataFrame

## ステップ1
http = urllib3.PoolManager()
url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13102&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1"
response = http.request('GET', url)
soup = BeautifulSoup(response.data, "html.parser")

## ステップ2
# ページ数を取得
s = soup.find("div", {"class": "pagination pagination_set-nav"})
num_pages = int(s.find_all("a")[-2].string)

# 全てのページのURLを作成
urls = []
urls.append(url)
for i in range(num_pages-1):
    pg = str(i+2)
    url_page = url + '&pn=' + pg
    urls.append(url_page)
    





names = [] #マンション名
addresses = [] #住所
locations0 = [] #立地1つ目（最寄駅/徒歩~分）
locations1 = [] #立地2つ目（最寄駅/徒歩~分）
locations2 = [] #立地3つ目（最寄駅/徒歩~分）
ages = [] #築年数
heights = [] #建物高さ
floors = [] #階
rent = [] #賃料
admin = [] #管理費
others = [] #敷/礼/保証/敷引,償却
floor_plans = [] #間取り
areas = [] #専有面積
detail_urls = [] # 詳細URL

## ステップ4
for url in urls:
    time.sleep(3)
    response = http.request('GET', url)
    soup = BeautifulSoup(response.data, "html.parser")

    apartments = soup.find_all('div', class_='cassetteitem')

## ステップ5
    for apartment in apartments:

        room_number = len(apartment.find_all('tbody'))

        name = apartment.find('div', class_='cassetteitem_content-title').text
        address = apartment.find('li', class_='cassetteitem_detail-col1').text

        for i in range(room_number):
            names.append(name)
            addresses.append(address)

## ステップ6
        sublocation = apartment.find('li', class_='cassetteitem_detail-col2')
        cols = sublocation.find_all('div')
        for i in range(len(cols)):
            text = cols[i].find(text=True)
            for j in range(room_number):
                if i == 0:
                    locations0.append(text)
                elif i == 1:
                    locations1.append(text)
                elif i == 2:
                    locations2.append(text)

## ステップ7
        age_and_height = apartment.find('li', class_='cassetteitem_detail-col3')
        age = age_and_height('div')[0].text
        height = age_and_height('div')[1].text

        for i in range(room_number):
            ages.append(age)
            heights.append(height)

## ステップ8
        table = apartment.find('table')
        rows = []
        rows.append(table.find_all('tr'))

        data = []
        for row in rows:
            for tr in row:
                cols = tr.find_all('td')
                if len(cols) != 0:
                    _floor = cols[2].text
                    _floor = re.sub('[\r\n\t]', '', _floor)

                    _rent_cell = cols[3].find('ul').find_all('li')
                    _rent = _rent_cell[0].find('span').text
                    _admin = _rent_cell[1].find('span').text

                    _deposit_cell = cols[4].find('ul').find_all('li')
                    _deposit = _deposit_cell[0].find('span').text
                    _reikin = _deposit_cell[1].find('span').text
                    _others = _deposit + '/' + _reikin

                    _floor_cell = cols[5].find('ul').find_all('li')
                    _floor_plan = _floor_cell[0].find('span').text
                    _area = _floor_cell[1].find('span').text

                    _detail_url = cols[8].find('a')['href']
                    _detail_url = 'https://suumo.jp' + _detail_url

                    text = [_floor, _rent, _admin, _others, _floor_plan, _area, _detail_url]
                    data.append(text)

        for row in data:
            floors.append(row[0])
            rent.append(row[1])
            admin.append(row[2])
            others.append(row[3])
            floor_plans.append(row[4])
            areas.append(row[5])
            detail_urls.append(row[6])
        print(data)

## ステップ9
#各リストをシリーズ化
names = Series(names)
addresses = Series(addresses)
locations0 = Series(locations0)
locations1 = Series(locations1)
locations2 = Series(locations2)
ages = Series(ages)
heights = Series(heights)
floors = Series(floors)
rent = Series(rent)
admin = Series(admin)
others = Series(others)
floor_plans = Series(floor_plans)
areas = Series(areas)
detail_urls = Series(detail_urls)

suumo_df = pd.concat([names, addresses, locations0, locations1, locations2, ages, heights, floors, rent, admin, others, floor_plans, areas, detail_urls], axis=1)

suumo_df.columns=['マンション名','住所','立地1','立地2','立地3','築年数','建物の高さ','階層','賃料料','管理費', '敷/礼/保証/敷引,償却','間取り','専有面積', '詳細URL']

suumo_df.to_csv('suumo_cyuoku.csv', sep = '\t',encoding='utf-16')

/Users/kai/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/kai/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[['4階', '10.4万円', '12000円', '10.4万円/-', 'ワンルーム', '32.92m2', 'https://suumo.jp/chintai/jnc_000047505747/?bc=100148802254'], ['4階', '11万円', '12000円', '11万円/-', 'ワンルーム', '32.77m2', 'https://suumo.jp/chintai/jnc_000040125469/?bc=100146064214'], ['4階', '11万円', '12000円', '11万円/-', 'ワンルーム', '32.77m2', 'https://suumo.jp/chintai/jnc_000047505745/?bc=100146716725'], ['7階', '11.9万円', '12000円', '11.9万円/-', '1DK', '32.92m2', 'https://suumo.jp/chintai/jnc_000047505748/?bc=100145784414'], ['11階', '12.2万円', '12000円', '12.2万円/-', '1DK', '32.77m2', 'https://suumo.jp/chintai/jnc_000047505740/?bc=100144314658'], ['4階', '11万円', '24000円', '11万円/-', 'ワンルーム', '32.77m2', 'https://suumo.jp/chintai/jnc_000047505744/?bc=100143996163'], ['13階', '12.3万円', '12000円', '12.3万円/-', '1DK', '32.77m2', 'https://suumo.jp/chintai/jnc_000047505742/?bc=100148113467'], ['11階', '12.2万円', '24000円', '12.2万円/-', '1DK', '32.77m2', 'https://suumo.jp/chintai/jnc_000047505741/?bc=100143996140'], ['2階', '14.7万円', '13000円', '14.7万円/-', '

[['16階', '17.6万円', '10000円', '35.2万円/17.6万円', '1DK', '47.57m2', 'https://suumo.jp/chintai/jnc_000045714299/?bc=100145848974'], ['15階', '18.3万円', '10000円', '36.6万円/18.3万円', '1DK', '47.57m2', 'https://suumo.jp/chintai/jnc_000046475471/?bc=100148483594'], ['11階', '18.1万円', '15000円', '36.2万円/18.1万円', '1LDK', '52.32m2', 'https://suumo.jp/chintai/jnc_000046475470/?bc=100145416613'], ['11階', '18.1万円', '15000円', '36.2万円/18.1万円', '1LDK', '52.32m2', 'https://suumo.jp/chintai/jnc_000046475469/?bc=100147764341'], ['17階', '18.2万円', '15000円', '36.4万円/18.2万円', '1K', '50.59m2', 'https://suumo.jp/chintai/jnc_000013340128/?bc=100140143871'], ['20階', '19.2万円', '10000円', '38.4万円/19.2万円', '1LDK', '46.38m2', 'https://suumo.jp/chintai/jnc_000045714300/?bc=100145393848'], ['20階', '19.2万円', '10000円', '38.4万円/19.2万円', '1DK', '46.38m2', 'https://suumo.jp/chintai/jnc_000047435737/?bc=100148985092'], ['5階', '19万円', '15000円', '38万円/19万円', '1LDK', '52.98m2', 'https://suumo.jp/chintai/jnc_000047352708/?bc=10014874394

/Users/kai/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[['2階', '25.4万円', '10000円', '50.8万円/25.4万円', '2LDK', '62.53m2', 'https://suumo.jp/chintai/jnc_000047462475/?bc=100149333196'], ['2階', '25.4万円', '15000円', '50.8万円/25.4万円', '2LDK', '62.53m2', 'https://suumo.jp/chintai/jnc_000047477599/?bc=100149128959'], ['9階', '32万円', '-', '32万円/32万円', '2LDK', '67.07m2', 'https://suumo.jp/chintai/jnc_000047462476/?bc=100146495648']]
[['2階', '25.4万円', '10000円', '50.8万円/25.4万円', '2LDK', '62.53m2', 'https://suumo.jp/chintai/jnc_000047477600/?bc=100149143412']]
[['3階', '10.8万円', '9000円', '10.8万円/16.2万円', '1DK', '25.66m2', 'https://suumo.jp/chintai/jnc_000045774556/?bc=100149104735'], ['10階', '19.5万円', '14500円', '19.5万円/29.25万円', '2LDK', '41.33m2', 'https://suumo.jp/chintai/jnc_000046379791/?bc=100149104905'], ['10階', '19.55万円', '14500円', '19.55万円/19.55万円', '2DK', '41.33m2', 'https://suumo.jp/chintai/jnc_000046240670/?bc=100148354005']]
[['7階', '18.8万円', '10000円', '18.8万円/18.8万円', '1LDK', '50.34m2', 'https://suumo.jp/chintai/jnc_000046636614/?bc=100146206951

...

[['2階', '8.8万円', '7000円', '8.8万円/8.8万円', 'ワンルーム', '21.78m2', 'https://suumo.jp/chintai/jnc_000046607790/?bc=100145875021'], ['6階', '9.1万円', '7000円', '9.1万円/9.1万円', '1K', '22.22m2', 'https://suumo.jp/chintai/jnc_000046814875/?bc=100146672331']]
[['6階', '11万円', '10000円', '22万円/11万円', 'ワンルーム', '33.4m2', 'https://suumo.jp/chintai/jnc_000045028015/?bc=100139362117']]
[['3階', '15.9万円', '12000円', '15.9万円/15.9万円', 'ワンルーム', '40.69m2', 'https://suumo.jp/chintai/jnc_000042916299/?bc=100131189981'], ['7階', '19.5万円', '15000円', '19.5万円/19.5万円', 'ワンルーム', '56.96m2', 'https://suumo.jp/chintai/jnc_000044476531/?bc=100137446610']]
[['11階', '28万円', '10000円', '28万円/-', 'ワンルーム', '51.93m2', 'https://suumo.jp/chintai/jnc_000046365645/?bc=100129191173']]
[['10階', '14万円', '-', '14万円/14万円', 'ワンルーム', '29.87m2', 'https://suumo.jp/chintai/jnc_000045669194/?bc=100141487173']]
[['3階', '8.2万円', '-', '8.2万円/8.2万円', '1K', '24.66m2', 'https://suumo.jp/chintai/jnc_000025613778/?bc=100129755512']]
[['3階', '8.1万円', '-', '8.

/Users/kai/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[['2階', '8.25万円', '5500円', '8.25万円/8.25万円', '1DK', '25.5m2', 'https://suumo.jp/chintai/jnc_000046774404/?bc=100146293945']]
[['2階', '8.25万円', '5500円', '8.25万円/8.25万円', '1DK', '25.5m2', 'https://suumo.jp/chintai/jnc_000045372772/?bc=100147951234']]
[['7階', '6.7万円', '-', '6.7万円/6.7万円', 'ワンルーム', '16.45m2', 'https://suumo.jp/chintai/jnc_000043835041/?bc=100134824716'], ['5階', '7.8万円', '-', '7.8万円/7.8万円', 'ワンルーム', '19.93m2', 'https://suumo.jp/chintai/jnc_000043705331/?bc=100130379605']]
[['3階', '8万円', '5000円', '8万円/8万円', 'ワンルーム', '21.45m2', 'https://suumo.jp/chintai/jnc_000046527916/?bc=100147881971']]
[['4階', '8.8万円', '8000円', '8.8万円/8.8万円', 'ワンルーム', '28m2', 'https://suumo.jp/chintai/jnc_000047254596/?bc=100146012838']]
[['8階', '6.5万円', '-', '13万円/6.5万円', 'ワンルーム', '15.22m2', 'https://suumo.jp/chintai/jnc_000032366599/?bc=100142506968'], ['2階', '8.5万円', '10000円', '8.5万円/8.5万円', '1DK', '30.57m2', 'https://suumo.jp/chintai/jnc_000046568052/?bc=100137300011']]
[['10階', '6.9万円', '8000円', '6.9万円

/Users/kai/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


[['10階', '5.9万円', '6000円', '5.9万円/5.9万円', 'ワンルーム', '17.41m2', 'https://suumo.jp/chintai/jnc_000047013126/?bc=100147601241'], ['10階', '7.5万円', '-', '7.5万円/7.5万円', 'ワンルーム', '22.31m2', 'https://suumo.jp/chintai/jnc_000046643909/?bc=100147098931']]
[['5階', '12.7万円', '8000円', '12.7万円/19.05万円', '1K', '27.29m2', 'https://suumo.jp/chintai/jnc_000046152776/?bc=100144060437']]
[['11階', '7万円', '-', '7万円/7万円', 'ワンルーム', '15.79m2', 'https://suumo.jp/chintai/jnc_000046593794/?bc=100145839259'], ['3階', '8万円', '5000円', '8万円/8万円', 'ワンルーム', '14.77m2', 'https://suumo.jp/chintai/jnc_000033353668/?bc=100033550758']]
[['2階', '8.5万円', '5000円', '8.5万円/8.5万円', '1K', '19.29m2', 'https://suumo.jp/chintai/jnc_000043214385/?bc=100147365324']]
[['2階', '5.9万円', '2000円', '-/-', 'ワンルーム', '12.32m2', 'https://suumo.jp/chintai/jnc_000044260598/?bc=100136616604'], ['5階', '5.9万円', '2000円', '-/-', 'ワンルーム', '12.04m2', 'https://suumo.jp/chintai/jnc_000046607797/?bc=100145863683']]
[['5階', '7万円', '4000円', '7万円/7万円', 'ワンルーム', '1